In [1]:
'''
2022/04/26 17:27
北大人民医院--瑞舒伐他汀vatin：政策性数据处理。
换药率和初始治疗选择
    2018.03.23-2019.03.22政策前
    2019.03.23-2020.03.22政策后
'''

'\n2022/04/24 17:27\n北大人民医院--瑞舒伐他汀vatin：依从性数据处理\n'

# 政策性性数据预处理

## 导入程序包

In [4]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np
import datetime

import math
import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\Rosu_Beijing_renmin_hospital


## 导入预定义函数

In [5]:
# 判断是否为float类型
def judge_float(x):
    try:
        a=float(x)
        return a
    except :
        return np.NaN

In [4]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [5]:
# 过滤特殊字符
def filter_spec_character(x):
    if pd.isnull(x):
        return np.nan
    else:
        x = re.sub('<|>|＞|＜|\+|\/|-|cm|kg','',str(x))
        return x

In [6]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else np.nan)
    df=df[df[feature].notnull()]
    return df

# 读取raw_data

## 用药数据预处理

In [113]:
'''
统计患者京新、可定、其他品牌他汀、其他降脂药的首末次处方时间、用药天数、用药剂量
注意先不需要分冠心病，首先落脚点是在高脂血症，即只要使用了他汀类药物的患者都纳入，当明确诊断为冠心病时则直接归为极高危人群，在后面的分层分析中考虑
'''

'\n统计患者京新、可定、其他品牌他汀、其他降脂药的首末次处方时间、用药天数、用药剂量\n注意先不需要分冠心病，首先落脚点是在高脂血症，即只要使用了他汀类药物的患者都纳入，当明确诊断为冠心病时则直接归为极高危人群，在后面的分层分析中考虑\n'

### 人民医院

In [133]:
# 人民医院用药
df_drug_order_1 = pd.read_excel(r'./data/raw_data/人民医院/4-药物医嘱.xlsx')

In [115]:
# 删除重复用药
df_drug_order_1 = df_drug_order_1.drop_duplicates(df_drug_order_1.columns,keep='first')
# 删除开药总数量为负的，退药记录
df_drug_order_1 = df_drug_order_1[df_drug_order_1['开药总数量']>=0]

In [116]:
# 人民医院--他汀用药
df_drug_vatin_1 = df_drug_order_1[df_drug_order_1.药物通用名称.str.contains('他汀')]

In [117]:
print(df_drug_order_1.shape)
print(df_drug_vatin_1.shape)

(422162, 22)
(251698, 22)


In [118]:
df_drug_vatin_1.columns

Index(['患者编号', '当前就诊次数/命中就诊总次数', '长期或临时', '药物编码', '药物商品名', '药物通用名称', '批准文号',
       '医嘱开始时间', '医嘱停止时间', 'ATC分类码', 'ATC分类名称', 'DDD值', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '用药途径名称', '开药总数量', '开药总数量单位名称', '药物剂型', '下嘱科室名称',
       '数据来源'],
      dtype='object')

In [119]:
# 提取有效字段
df_drug_vatin_1 = df_drug_vatin_1[['患者编号', '药物编码', '药物通用名称','医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型']]

In [120]:
df_drug_vatin_1.shape

(251698, 11)

In [121]:
df_drug_vatin_1.head()

,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,开药总数量单位名称,药物剂型
0,pat_0d9297a346707bce01f0b4637484efe9,瑞舒伐他汀钙,2019-12-31 07:54:51,NaN,旧 10mgx28片/盒,1.0,片,QD(8am),1.0,盒,片剂
1,pat_0ebd83a8c3990a103016a650f84616f8,瑞舒伐他汀钙,2018-03-11 11:23:20,NaN,可定 10mg x7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
2,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-01-08 16:29:39,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
3,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-03-22 15:00:58,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂
4,pat_13da443c402ad02c9e9de0c5c0751eda,阿托伐他汀钙,2018-06-26 14:08:42,NaN,旧 10mgx7片/盒,10.0,mg,QD(8am),5.0,盒,片剂


In [122]:
df_drug_vatin_1.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.1_人民_drug_vatin.csv')

### 世纪坛医院

In [132]:
# 世纪坛用药
df_drug_order_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                           sheet_name='用药')

In [158]:
# 世纪坛医院-他汀用药
df_drug_order_2 = df_drug_order_2[df_drug_order_2['药品名称'].notnull()]
# 删除重复
df_drug_order_2 = df_drug_order_2.drop_duplicates(df_drug_order_2.columns,keep='first')
# 他汀类药物
df_drug_vatin_2 = df_drug_order_2[df_drug_order_2.药品名称.str.contains('他汀|可定|阿乐|立普妥')]

In [ ]:
print(df_drug_order_2.shape)
print(df_drug_vatin_2.shape)

In [ ]:
df_drug_vatin_2.columns

In [ ]:
# 提取有效字段
df_drug_vatin_2 = df_drug_vatin_2[['patient_SN', '就诊时间（门诊）','给药开始时间', '给药停止时间', '用药时长',
                '用药频率', '单次剂量','药品类别', '用药（商品名称）', '药物规格', '药品医嘱内容', '药品名称']]

In [ ]:
# 世纪坛医院的用药频率和药品类别部分数据混乱
# 就诊时间（门诊）-补充-给药开始时间缺失值
df_drug_vatin_2=df_drug_vatin_2.reset_index(drop=True)
for i in range(df_drug_vatin_2.shape[0]):
    x =  df_drug_vatin_2.loc[i,'药品类别']
    if re.match('.*?次', x):
        df_drug_vatin_2.loc[i,'用药频率'] = x
    t = df_drug_vatin_2.loc[i,'就诊时间（门诊）']
    if pd.notnull(t):
        df_drug_vatin_2.loc[i,'给药开始时间'] = t

In [ ]:
df_drug_vatin_2.columns

In [ ]:
# 有效字段排序
df_drug_vatin_2 = df_drug_vatin_2[['patient_SN', '药品名称', '给药开始时间', '给药停止时间', '用药时长', '药物规格', '单次剂量', '用药频率']]
# 修改列名
df_drug_vatin_2.columns = ['患者编号', '药物通用名称','医嘱开始时间', '医嘱停止时间', '用药时长', '药物规格', '药物剂量', '频率名称']

In [ ]:
# 删除医嘱开始时间为空的数据
df_drug_vatin_2 = df_drug_vatin_2[df_drug_vatin_2.医嘱开始时间.notnull()]

In [ ]:
df_drug_vatin_2.shape

In [ ]:
df_drug_vatin_2.head()

In [ ]:
df_drug_vatin_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.2_世纪坛_drug_vatin.csv')

### 合并用药数据

In [ ]:
# 添加数据来源
df_drug_vatin_1.insert(0,'data_source','人民医院')
df_drug_vatin_2.insert(0,'data_source','世纪坛医院')

In [ ]:
# 合并人民医院和世纪坛医院的他汀类用药
df_drug_vatin = pd.concat([df_drug_vatin_1, df_drug_vatin_2],axis=0).reset_index(drop=True)

In [ ]:
df_drug_vatin.shape

In [ ]:
df_drug_vatin.data_source.value_counts()

In [ ]:
df_drug_vatin.head()

In [ ]:
df_drug_vatin.to_csv(project_path+'/data/pre_processed_raw_data/df_2.3.3_他汀用药.csv')

### 提取瑞舒伐他汀

In [ ]:
# 提取瑞舒伐他汀
df_drug_resuvatin = df_drug_vatin[df_drug_vatin.药物通用名称.str.contains('瑞舒|可定')]

In [ ]:
df_drug_resuvatin.columns

In [ ]:
# 用药时长和药物规格部分数据混乱
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
# 药物规格drug specifications
for i in range(df_drug_resuvatin.shape[0]):
    s = df_drug_resuvatin.loc[i,'用药时长']
    if re.match('.*?[盒片]', str(s)):
        df_drug_resuvatin.loc[i,'药物规格'] = df_drug_resuvatin.loc[i,'药物规格'] + s

In [ ]:
df_drug_resuvatin.shape

In [ ]:
# 删除重复
df_drug_resuvatin = df_drug_resuvatin.drop_duplicates(df_drug_resuvatin.columns,keep='first')

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.to_csv(project_path + '/data/processed_data/df_2.3.4_瑞舒伐他汀用药.csv')

In [ ]:
# 其它他汀药物
df_drug_vatin_other = df_drug_vatin[~ df_drug_vatin.药物通用名称.str.contains('瑞舒|可定')]
# 删除重复数据
df_drug_vatin_other = df_drug_vatin_other.drop_duplicates(df_drug_vatin_other.columns, keep='first')

In [ ]:
df_drug_vatin_other.shape

In [ ]:
df_drug_vatin_other.to_csv(project_path + '/data/processed_data/df_2.3.4_其它他汀用药.csv')

In [ ]:
df_drug_vatin_other.head(1)

### 打标签：京新、可定

In [ ]:
'''
进口瑞舒伐他汀钙（可定）药物编码为203238，包括3个规格：10mg/片、可定10mg/片、可定10mg×7片/盒。
国产瑞舒伐他汀钙（京新）药物编码为204979，包括4个规格：10mg×28片/盒、京新10mg×28片/盒、旧 10mg×28片/盒、京新10mg/片。
'''

In [ ]:
df_drug_resuvatin.药物规格.value_counts()

In [ ]:
df_drug_resuvatin.head(1)

In [ ]:
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    s = df_drug_resuvatin.loc[i,'药物规格']
    if re.match('京新', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '京新'
    elif re.match('可定', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'
    elif re.match('.*?28片', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '京新'
    elif re.match('.*?7片', s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'
    elif re.match('10毫克 /片',s):
        df_drug_resuvatin.loc[i,'resuvatin_brand'] = '可定'

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.5_打厂商标签.csv')

In [283]:
df_drug_resuvatin=pd.read_csv(project_path+'/data/processed_data/df_2.3.5_打厂商标签.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

### 计算日剂量

#### 转换频率

In [284]:
df_drug_resuvatin.频率名称.unique()

array(['QD(8am)', '1/晚(10PM)', 'QN每晚 (8pm)', 'BID(8-4)', 'ONCE',
       'TID(8-12-4)', 'Q12H (8-8)', 'QD1(8am)', 'QOD5 （8pm）', 'QOD(8am)',
       'QD3(12N)', 'QD6(4pm)', 'QOD1(12N)', 'BID1 (8-4)口服', '必要时',
       'QD4 (5pm)', '05 每晚一次', '01 每日一次', '03 每日三次', '02 每日两次'],
      dtype=object)

In [285]:
# 清洗频次
half=['QOD5 （8pm）','QOD(8am)','QOD1(12N)',]
one=['QD(8am)','1/晚(10PM)', 'QN每晚 (8pm)','QD1(8am)','ONCE','01 每日一次','QD3(12N)', 'QD6(4pm)','QD4 (5pm)']
two=['BID(8-4)','Q12H (8-8)','02 每日两次','BID1 (8-4)口服',]
three=['TID(8-12-4)','03 每日三次',]

df_drug_resuvatin['频率名称']=df_drug_resuvatin['频率名称'].apply(lambda x: 0.5 if x in half else
                                                                          1 if x in one else
                                                                          2 if x in two else
                                                                          3 if x in three else np.nan)

In [286]:
df_drug_resuvatin.频率名称.value_counts()

1.0    56541
2.0      891
3.0       39
0.5       26
Name: 频率名称, dtype: int64

In [287]:
# 删除频率为空的用药
df_drug_resuvatin = df_drug_resuvatin[df_drug_resuvatin.频率名称.notnull()]

In [288]:
df_drug_resuvatin.shape

(57497, 14)

#### 单次剂量处理

In [ ]:
df_drug_resuvatin.药物剂量单位名称.value_counts()

In [ ]:
# 片--10mg；盒--280mg
df_drug_resuvatin= df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    u = df_drug_resuvatin.loc[i,'药物剂量单位名称']
    # 将数值为1的剂量统一换成10mg
    if df_drug_resuvatin.loc[i,'药物剂量']== 1:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif df_drug_resuvatin.loc[i,'药物剂量']==14:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif df_drug_resuvatin.loc[i,'药物剂量']== 28:
        df_drug_resuvatin.loc[i,'药物剂量']=10
    elif pd.isnull(u):
        continue
    elif re.match('片',u):
        df_drug_resuvatin.loc[i,'药物剂量']= df_drug_resuvatin.loc[i,'药物剂量']*10

In [ ]:
# 将超过50mg的单次剂量重置为10mg
df_drug_resuvatin['药物剂量'] = df_drug_resuvatin['药物剂量'].apply(lambda x: 10 if x>50 else x)

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.药物剂量.value_counts()

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.6.2_单次剂量处理.csv')

#### 计算日剂量

In [249]:
df_drug_resuvatin['日剂量']=df_drug_resuvatin['药物剂量'].astype('float') * df_drug_resuvatin['频率名称'].astype('float')

In [ ]:
# 限定瑞舒伐他汀最大日剂量，将超过50mg的日剂量重置为单次剂量
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    if df_drug_resuvatin.loc[i,'日剂量'] > 50:
        df_drug_resuvatin.loc[i,'日剂量']=df_drug_resuvatin.loc[i,'药物剂量']

In [ ]:
df_drug_resuvatin['日剂量'].value_counts()

In [ ]:
df_drug_resuvatin.shape

In [ ]:
df_drug_resuvatin.head(1)

In [ ]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.3.6.3_计算日计量.csv')

### 删除重复用药

In [7]:
'''
重复数据、多次开药，取最后一次开药记录
注意，别删掉不同brand的用药，后面需要纳排
'''

'\n重复数据、多次开药，取最后一次开药记录\n注意，别删掉不同brand的用药，后面需要纳排\n'

In [12]:
df_drug_resuvatin=pd.read_csv(project_path+'/data/processed_data/df_2.3.6.3_计算日计量.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df_drug_resuvatin.shape

(57497, 15)

In [14]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量'],
      dtype='object')

In [15]:
# 同一天多次用药记录，取最后一次;
df_drug_resuvatin['医嘱开始日期'] = df_drug_resuvatin.医嘱开始时间.apply(lambda x: str(x).split(' ')[0])
df_drug_resuvatin = df_drug_resuvatin.sort_values(['患者编号','医嘱开始时间'],ascending=True)
df_drug_resuvatin = df_drug_resuvatin.drop_duplicates(subset=['患者编号','resuvatin_brand','医嘱开始日期'],keep='last')

In [16]:
df_drug_resuvatin.shape

(57301, 16)

In [13]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.7_删除用药重复记录.csv')

### 计算用药时长和开药总剂量

In [18]:
'''
人民医院：用药时长 = 开药总剂量/日剂量；医嘱结束时间 = 医嘱开始时间 + 用药时长
世纪坛医院：用药时长 = 用药停止时间-用药开始时间 + 1
'''
'''
人民医院：开药总剂量=开药总数量*盒数
世纪坛医院：开药总剂量=日剂量*用药时长
'''

'\n人民医院：开药总剂量=开药总数量*盒数\n世纪坛医院：开药总剂量=日剂量*用药时长\n'

#### 计算世纪坛--用药时长

In [19]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期'],
      dtype='object')

In [20]:
df_drug_resuvatin.shape

(57301, 16)

In [34]:
# 对于世纪坛医院用药结束时间为空的数据，将下一次开药时间作为其上一次结束时间。
temp_list=[]
for i in df_drug_resuvatin.患者编号.unique():
    temp = df_drug_resuvatin[df_drug_resuvatin['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    hospital = temp.loc[0,'data_source']
    if re.match('世纪坛医院',hospital) and (temp.shape[0]>1):
        for j in range(temp.shape[0]):
            # 医嘱结束时间为空
            if pd.isnull(temp.loc[j,'医嘱停止时间']) and (j+1<temp.shape[0]):
                # 提取下一次用药记录
                temp_end = temp[temp['医嘱开始日期'] > temp.loc[j,'医嘱开始日期']].reset_index(drop=True)
                if temp_end.shape[0]>0:
                    temp.loc[j,'医嘱停止时间'] = temp_end.loc[0,'医嘱开始时间']
    temp_list.append(temp)

In [36]:
df_drug_resuvatin=pd.concat(temp_list,axis=0)
del temp_list

In [37]:
df_drug_resuvatin.shape

(57301, 16)

In [38]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_temp_补充用药结束时间.csv')

In [39]:
# 用药时长=医嘱结束时间-医嘱开始时间+1
df_drug_resuvatin['用药时长']=np.nan
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    # 根据医嘱时间计算
    drug_start_time = df_drug_resuvatin.loc[i,'医嘱开始时间']
    drug_end_time = df_drug_resuvatin.loc[i,'医嘱停止时间']
    hospital= df_drug_resuvatin.loc[i,'data_source']
    if re.match('世纪坛医院', hospital):
        df_drug_resuvatin.loc[i,'用药时长'] = (pd.to_datetime(drug_end_time)-pd.to_datetime(drug_start_time)).days + 1
#     # 根据开药剂量计算
#     prescription_sum = df_drug_resuvatin.loc[i,'开药总剂量']
#     dosage = df_drug_resuvatin.loc[i,'日剂量']
#     if pd.isnull(prescription_sum):
#         prescription_time = np.nan
#     else:
#         prescription_time = math.ceil(prescription_sum / dosage) # 向上取整
#     df_drug_resuvatin.loc[i,'用药时长']= max(-1,drug_time,prescription_time)

In [40]:
df_drug_resuvatin.shape

(57301, 16)

In [41]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.1_计算世纪坛_用药时长.csv')

#### 计算开药总剂量

In [42]:
# 盒--280mg；片--10mg
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    prescription_u = df_drug_resuvatin.loc[i,'开药总数量单位名称']
    # 世纪坛医院：日剂量*用药时长
    if pd.isnull(prescription_u):
        df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'用药时长'] * df_drug_resuvatin.loc[i,'日剂量']
    # 人民医院: 开药总剂量
    elif re.match('盒', prescription_u):
        # 京新一盒280mg；可定一盒70mg
        brand = df_drug_resuvatin.loc[i,'resuvatin_brand']
        if re.match('京新', brand):
            df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*280
        else:
            df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*70
    elif re.match('片', prescription_u):
        df_drug_resuvatin.loc[i,'开药总剂量']= df_drug_resuvatin.loc[i,'开药总数量']*10

In [43]:
# 删除数值为0的开药总剂量
df_drug_resuvatin = df_drug_resuvatin[df_drug_resuvatin['开药总剂量']>0]

In [44]:
df_drug_resuvatin.shape

(54471, 17)

In [47]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.2计算开药总剂量.csv')

PermissionError: [Errno 13] Permission denied: 'D:\\PycharmProjects\\Rosu_Beijing_renmin_hospital/data/processed_data/df_2.1.8.2计算开药总剂量.csv'

#### 计算人民医院医嘱结束时间

In [49]:
'''
人民：医嘱停止时间 = 医嘱开始时间 + 用药时长
'''

'\n人民：医嘱停止时间 = 医嘱开始时间 + 用药时长\n'

In [50]:
df_drug_resuvatin = df_drug_resuvatin.reset_index(drop=True)
for i in range(df_drug_resuvatin.shape[0]):
    # 计算人民医院：用药时长
    if df_drug_resuvatin.loc[i,'data_source'] == '人民医院':
        d = round(df_drug_resuvatin.loc[i,'开药总剂量']/df_drug_resuvatin.loc[i,'日剂量'],0)
        df_drug_resuvatin.loc[i,'用药时长']= d
        df_drug_resuvatin.loc[i,'医嘱停止时间'] = pd.to_datetime(df_drug_resuvatin.loc[i,'医嘱开始时间']) + datetime.timedelta(days=d)

In [51]:
df_drug_resuvatin.shape

(54471, 17)

In [52]:
df_drug_resuvatin.to_csv(project_path+'/data/processed_data/df_2.1.8.3_计算人民医院_医嘱结束时间.csv')

## 诊断数据预处理

### 人民医院

In [8]:
# 人民医院--诊断数据
df_diagnose_1 = pd.read_excel(r'./data/raw_data/人民医院/3-诊断.xlsx')
df_diagnose_supplement = pd.read_excel(r'./data/raw_data/人民医院/诊断-补.xlsx')

df_diagnose_1 = pd.concat([df_diagnose_1, df_diagnose_supplement],axis=0).drop_duplicates(subset=df_diagnose_1.columns, keep='first')

In [9]:
df_diagnose_1.shape

(1005384, 8)

In [10]:
df_diagnose_1.columns

Index(['患者编号', '当前就诊次数/命中就诊总次数', '诊断编码', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称',
       '数据来源'],
      dtype='object')

In [11]:
# 提取有效字段
df_diagnose_1 = df_diagnose_1[['患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称']]
# 删除重复数据
df_diagnose_1 = df_diagnose_1.drop_duplicates(df_diagnose_1.columns, keep='first')

In [14]:
# 患者限定
df_diagnose_1 = df_diagnose_1[df_diagnose_1.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [15]:
df_diagnose_1.shape

(231391, 5)

In [16]:
df_diagnose_1.head()

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
0,pat_0d9297a346707bce01f0b4637484efe9,高胆固醇血症,2019-12-31 07:54:05,否,门诊诊断
1,pat_0ebd83a8c3990a103016a650f84616f8,高胆固醇血症,2018-03-11 11:06:21,否,门诊诊断
121,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断
294,pat_04dfcad1038b64914a0cb3dbe835ae57,冠心病,2018-07-20 15:48:42,否,门诊诊断
295,pat_04dfcad1038b64914a0cb3dbe835ae57,高脂血症,2018-07-20 15:48:01,否,门诊诊断


In [ ]:
df_diagnose_1.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.1_人民医院_诊断.csv')

### 世纪坛医院

In [17]:
# 世纪坛医院--诊断数据
df_diagnose_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                              sheet_name='诊断')

In [18]:
df_diagnose_2.shape

(617655, 8)

In [19]:
df_diagnose_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '就诊次数（住院）', '就诊时间（门诊）',
       '就诊科室（门诊）', 'ICD10编码', 'ICD10名称'],
      dtype='object')

In [20]:
# 提取有效字段
df_diagnose_2 = df_diagnose_2[['patient_SN','ICD10名称','就诊时间（门诊）']]
# 删除重复数据
df_diagnose_2 = df_diagnose_2.drop_duplicates(df_diagnose_2.columns, keep='first')

In [23]:
'患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称'

('患者编号', '诊断名称', '诊断日期', '是否主要诊断', '诊断类别名称')

In [24]:
df_diagnose_2.columns = ['患者编号', '诊断名称', '诊断日期']

In [25]:
# 患者限定
df_diagnose_2 = df_diagnose_2[df_diagnose_2.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [26]:
df_diagnose_2.head(1)

,患者编号,诊断名称,诊断日期
234,0020640c4052f1dae97fcf39be390e1b,I25.105,NaT


In [ ]:
df_diagnose_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.2_世纪坛_诊断.csv')

In [64]:
df_diagnose_2=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.4.2_世纪坛_诊断.csv')
if 'Unnamed: 0' in df_diagnose_2.columns:
    del df_diagnose_2['Unnamed: 0']

In [65]:
df_diagnose_2.head()

,患者编号,诊断名称,诊断日期
0,0000706397b506f3ddc1a470b0e5505a,前列腺增生,2018-05-24
1,0000706397b506f3ddc1a470b0e5505a,高血压,2018-05-24
2,0000706397b506f3ddc1a470b0e5505a,高脂血症,2018-05-24
3,0000706397b506f3ddc1a470b0e5505a,消瘦,2018-05-24
4,0000706397b506f3ddc1a470b0e5505a,腹痛,2018-05-24


### 合并疾病诊断

In [27]:
# 合并人民医院和世纪坛医院的疾病诊断
df_diagnose = pd.concat([df_diagnose_1, df_diagnose_2],axis=0).reset_index(drop=True)

In [28]:
# 患者限定
df_diagnose = df_diagnose[df_diagnose.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [29]:
df_diagnose.shape

(303522, 5)

In [30]:
df_diagnose.head()

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
0,pat_0d9297a346707bce01f0b4637484efe9,高胆固醇血症,2019-12-31 07:54:05,否,门诊诊断
1,pat_0ebd83a8c3990a103016a650f84616f8,高胆固醇血症,2018-03-11 11:06:21,否,门诊诊断
2,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断
3,pat_04dfcad1038b64914a0cb3dbe835ae57,冠心病,2018-07-20 15:48:42,否,门诊诊断
4,pat_04dfcad1038b64914a0cb3dbe835ae57,高脂血症,2018-07-20 15:48:01,否,门诊诊断


In [31]:
df_diagnose.to_csv(project_path+'/data/pre_processed_raw_data/df_2.4.3_诊断.csv')

In [162]:
df_diagnose=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.4.3_诊断.csv')
if 'Unnamed: 0' in df_diagnose.columns:
    del df_diagnose['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 检验数据预处理

### 人民医院

In [32]:
# 人民医院--检验数据
df_check_1 = pd.read_excel(r'./data/raw_data/人民医院/检验.xlsx',
                          header=1)

In [33]:
df_check_1.shape

(359081, 15)

In [34]:
df_check_1.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', '检验子项结果正常标志',
       '检验子项中文名', '检验子项英文名', '检验子项结果', '检验子项结果数值', '检验子项单位'],
      dtype='object')

In [35]:
df_check_1.columns = ['患者编号', '当前就诊次数/命中就诊总次数', '检验单号', '检验项目名称', '采集时间', '采集时间_date',
       '采集时间_time', '标本名称', '标本类型', '检验子项结果正常标志', '检验子项中文名', '检验子项英文名',
       '检验子项结果', '检验子项结果数值', '检验子项单位']

In [36]:
# 提取有效字段
df_check_1 = df_check_1[['患者编号', '采集时间', '采集时间_date','采集时间_time', '检验子项结果正常标志', '检验子项英文名',
                         '检验子项结果','检验子项结果数值', '检验子项单位']]
# 修改列名
df_check_1 = df_check_1.rename(columns={'采集时间':'检验时间',
                                       '采集时间_date':'检验时间_date',
                                       '采集时间_time':'检验时间_time'})

In [37]:
# 修改项名
df_check_1['检验子项英文名'] = df_check_1['检验子项英文名'].apply(lambda x: 'BUN' if re.match('尿素氮',str(x)) else
                                                                       'Urea' if re.match('尿素',str(x)) else
                                                                       'Glu' if re.match('血糖',str(x)) else
                                                                       'TBIL' if re.match('总胆红素',str(x)) else
                                                                       'TG' if re.match('甘油三酯',str(x)) else
                                                                       'CK' if re.match('肌酸激酶',str(x)) else x)

In [38]:
df_check_1.head(1)

,患者编号,检验时间,检验时间_date,检验时间_time,检验子项结果正常标志,检验子项英文名,检验子项结果,检验子项结果数值,检验子项单位
0,pat_13da443c402ad02c9e9de0c5c0751eda,2018-04-06 07:11:20,2018-04-06,07:11:20,正常,Urea,4.96,4.96,mmol/L


In [39]:
# 删除重复数据
df_check_1 = df_check_1.drop_duplicates(df_check_1.columns, keep='first')
# 删除缺失值
df_check_1 = df_check_1[df_check_1.检验时间_date.notnull()]
df_check_1 = df_check_1[df_check_1['检验子项结果数值'].notnull() & df_check_1.检验子项英文名.notnull()]

In [40]:
df_check_1.shape

(352521, 9)

In [41]:
#删除只有一次的检验
df_check_1_count = df_check_1['患者编号'].value_counts().reset_index()
df_check_1_count = df_check_1_count[df_check_1_count['患者编号']>1]
# 提取剩余的检验
df_check_1 = df_check_1[df_check_1.患者编号.isin(df_check_1_count['index'].unique())]

In [42]:
df_check_1.shape

(352317, 9)

In [43]:
# 限定患者
df_check_1 = df_check_1[df_check_1.患者编号.isin(df_drug_resuvatin.患者编号.unique())]

In [44]:
df_check_1.shape

(82296, 9)

In [45]:
df_check_1.head()

,患者编号,检验时间,检验时间_date,检验时间_time,检验子项结果正常标志,检验子项英文名,检验子项结果,检验子项结果数值,检验子项单位
261,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,AST,29,29.00,U/L
262,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,CK,67,67.00,U/L
263,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,LDL-C,2.42,2.42,mmol/L
264,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,ALT,28,28.00,U/L
265,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16 08:55:23,2018-08-16,08:55:23,正常,Urea,5.34,5.34,mmol/L


In [46]:
# 判断是否为float类型
def judge_float(x):
    try:
        a=float(x)
        return a
    except :
        return np.NaN

In [47]:
# 转置表头
df_check_1 = df_check_1.reset_index(drop=True)
df_check_1_T = pd.DataFrame()
count = 0
for i in df_check_1.患者编号.unique():
    count += 1
    # 同一个病人ID的相同检验项
    temp = df_check_1[df_check_1['患者编号']== i].sort_values(['检验时间'],ascending=True)
    temp = temp.drop_duplicates(subset=['患者编号','检验时间_date','检验子项英文名'],keep='last').reset_index(drop=True)
    ## 检验项转置为表头
    if temp.shape[0]>0:
        temp['检验子项结果']=temp['检验子项结果'].apply(judge_float)
        temp= temp.pivot_table('检验子项结果', ['患者编号','检验时间_date'], '检验子项英文名').reset_index()
        df_check_1_T = pd.concat([df_check_1_T,temp],axis=0)

In [48]:
df_check_1_T.shape

(9890, 16)

In [49]:
df_check_1_T.head()

,患者编号,检验时间_date,ALT,AST,CHO,CK,HDL-C,LDL-C,TBIL,TG,Urea,apo-A1,apo-B,CK-MB,BUN,Glu
0,pat_2bb0dfeccae4278dbf399d6428a2573d,2018-08-16,28.0,29.0,3.87,67.0,1.10,2.42,8.5,2.89,5.34,119.5,86.4,NaN,NaN,NaN
1,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-04-10,17.0,16.0,2.82,72.0,0.95,1.43,6.4,1.78,6.21,112.9,60.7,NaN,NaN,NaN
2,pat_2bb0dfeccae4278dbf399d6428a2573d,2019-11-06,15.0,14.0,3.19,57.0,1.06,1.78,8.8,2.56,4.32,117.5,66.8,NaN,NaN,NaN
0,pat_043aba443d80d048105d8903d2034364,2018-06-05,30.0,29.0,4.78,114.0,1.30,3.28,14.8,1.48,6.67,NaN,NaN,NaN,NaN,NaN
0,pat_272f3f5730bfc5a361c461b4fde1dbb8,2018-08-15,16.0,24.0,4.25,133.0,1.24,2.22,11.8,1.15,8.29,125.7,79.2,NaN,NaN,NaN


In [ ]:
df_check_1_T.to_csv(project_path+'/data/pre_processed_raw_data/df_2.5.1_人民_检验.csv')

In [8]:
df_check_1_T=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.5.1_人民_检验.csv')
if 'Unnamed: 0' in df_check_1_T.columns:
    del df_check_1_T['Unnamed: 0']

### 世纪坛医院

In [50]:
# 世纪坛医院--检验数据
df_check_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                           sheet_name='检验')
df_check_2 = df_check_2[:-2]
df_check_2 = df_check_2.dropna(how='all',axis=1)

In [51]:
df_check_2.shape

(36126, 45)

In [52]:
df_check_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '就诊次数（住院）', '就诊时间（门诊）',
       '就诊科室（门诊）', '检验时间', '总胆固醇(TC)-静脉血', '天门冬氨酸氨基转移酶(AST)-静脉血',
       '丙氨酸氨基转移酶(ALT)-静脉血', '总胆红素(TBIL)-静脉血', '肌酐(Crea)-静脉血', '甘油三酯(TG)-静脉血',
       '尿素(Urea)-静脉血(空腹)', '尿素(Urea)-静脉血(空腹)-定性', '尿素(Urea)-静脉血',
       '肌酸激酶(CK)-静脉血(空腹)', '肌酸激酶(CK)-静脉血', '肌酸激酶(CK)-静脉血-定性', '葡萄糖(Glu)-末梢血',
       '葡萄糖(Glu)-静脉血(2小时)', '甘油三酯(TG)-静脉血(空腹)', '甘油三酯(TG)-静脉血(空腹)-定性',
       '甘油三酯(TG)-静脉血-定性', '总胆固醇(TC)-静脉血(空腹)', '总胆固醇(TC)-静脉血(空腹)-定性',
       '载脂蛋白B(apoB)-静脉血(空腹)', '载脂蛋白AI(apoAI)-静脉血(空腹)',
       '载脂蛋白B(apoB)-静脉血(空腹)-定性', '高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹)',
       '低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹)', '低密度脂蛋白胆固醇(LDL-C)-静脉血',
       '高密度脂蛋白胆固醇(HDL-C)-静脉血', '载脂蛋白AI(apoAI)-静脉血', '载脂蛋白B(apoB)-静脉血',
       '脂蛋白a(LPa)-静脉血', '脂蛋白a(LPa)-静脉血-定性', '尿蛋白定性-24小时尿液-定性', '微量总蛋白-24小时尿液',
       '总蛋白(TP)-尿液-定性', '总蛋白(TP)-尿液', '微量白蛋白(mAlb)-尿液', '葡萄糖(Glu)-尿液-定性',
       '肌酸激酶(CK)-静脉血(空腹)-定性', 'Unnamed: 67'],
      dtype='object')

In [53]:
# 删除无效列
df_check_2 = df_check_2.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 67','就诊次数（住院）',
                              '就诊时间（门诊）', '就诊科室（门诊）'],axis=1)
# 删除定性列
q_list = [i for i in df_check_2.columns if '定性' in i]
df_check_2 = df_check_2.drop(q_list,axis=1)
# 修改列名
df_check_2 = df_check_2.rename(columns={'patient_SN':'患者编号',
                                       '检验时间':'检验时间_date'})
df_check_2['检验时间_date'] = df_check_2['检验时间_date'].apply(lambda x: str(x).split(' ')[0])

In [54]:
# 删除重复列
df_check_2 = df_check_2.drop_duplicates(df_check_2.columns, keep='first')
# 删除只有一次的检验
df_check_2_count = df_check_2['患者编号'].value_counts().reset_index()
df_check_2_count = df_check_2_count[df_check_2_count['患者编号']>1]
# 提取剩余的检验
df_check_2 = df_check_2[df_check_2.患者编号.isin(df_check_2_count['index'].unique())]

In [55]:
df_check_2.shape

(32715, 28)

In [56]:
df_check_2.head()

,患者编号,检验时间_date,总胆固醇(TC)-静脉血,天门冬氨酸氨基转移酶(AST)-静脉血,丙氨酸氨基转移酶(ALT)-静脉血,总胆红素(TBIL)-静脉血,肌酐(Crea)-静脉血,甘油三酯(TG)-静脉血,尿素(Urea)-静脉血(空腹),尿素(Urea)-静脉血,...,高密度脂蛋白胆固醇(HDL-C)-静脉血(空腹),低密度脂蛋白胆固醇(LDL-C)-静脉血(空腹),低密度脂蛋白胆固醇(LDL-C)-静脉血,高密度脂蛋白胆固醇(HDL-C)-静脉血,载脂蛋白AI(apoAI)-静脉血,载脂蛋白B(apoB)-静脉血,脂蛋白a(LPa)-静脉血,微量总蛋白-24小时尿液,总蛋白(TP)-尿液,微量白蛋白(mAlb)-尿液
2,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-18,NaN,NaN,NaN,NaN,NaN,NaN,13.41,NaN,...,1.17,1.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-22,NaN,NaN,NaN,NaN,NaN,NaN,11.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,9.29,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0006b9e1f7ff71744d8162c0c71b2a95,2018-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_check_2.to_csv(project_path+'/data/pre_processed_raw_data/df_2.5.2_世纪坛_检验.csv')

In [9]:
df_check_2=pd.read_csv(project_path+'/data/pre_processed_raw_data/df_2.5.2_世纪坛_检验.csv')
if 'Unnamed: 0' in df_check_2.columns:
    del df_check_2['Unnamed: 0']

# 数据处理

In [14]:
'''
人口统计学特征、现病史和既往病史、药物使用情况、临床症状／体征、实验室检查和功能检查、患者治疗满意度
'''

'\n人口统计学特征、现病史和既往病史、药物使用情况、临床症状／体征、实验室检查和功能检查、患者治疗满意度\n'

In [277]:
df_drug_resuvatin = pd.read_csv(project_path+'/data/processed_data/df_2.1.8.3_计算人民医院_医嘱结束时间.csv')
if 'Unnamed: 0' in df_drug_resuvatin.columns:
    del df_drug_resuvatin['Unnamed: 0']

In [278]:
df_drug_resuvatin.shape

(54471, 17)

In [181]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期', '开药总剂量'],
      dtype='object')

In [ ]:
# ## 统计人数

# '''
# 政策前后只用京新或可定的人数，2019.3.22
# '''

# # 政策前后标签,0--政策前；1--政策后
# df_drug_resuvatin['strategy_label']=df_drug_resuvatin.医嘱开始时间.apply(lambda x: 0 if pd.to_datetime(x)<=pd.to_datetime('2019-03-22') else 1)

# # 提取政策前后数据
# df_drug_resuvatin_before=df_drug_resuvatin[df_drug_resuvatin['strategy_label']==0]
# df_drug_resuvatin_after=df_drug_resuvatin[df_drug_resuvatin['strategy_label']==1]

# print(df_drug_resuvatin_before.shape)
# print(df_drug_resuvatin_after.shape)

# # 只用一种药
# temp_list=[]
# for i in df_drug_resuvatin_before.患者编号.unique():
#     temp=df_drug_resuvatin_before[df_drug_resuvatin_before['患者编号']==i]
#     if temp.resuvatin_brand.nunique()==1:
#         temp_list.append(temp)
# df_drug_resuvatin_before=pd.concat(temp_list,axis=0)
# del temp_list

# temp_list=[]
# for i in df_drug_resuvatin_after.患者编号.unique():
#     temp=df_drug_resuvatin_after[df_drug_resuvatin_after['患者编号']==i]
#     if temp.resuvatin_brand.nunique()==1:
#         temp_list.append(temp)
# df_drug_resuvatin_after=pd.concat(temp_list,axis=0)
# del temp_list

# print(df_drug_resuvatin_before.shape)
# print(df_drug_resuvatin_after.shape)

# df_drug_resuvatin.患者编号.nunique()

# df_drug_resuvatin_before_京新=df_drug_resuvatin_before[df_drug_resuvatin_before['resuvatin_brand']=='京新']
# df_drug_resuvatin_before_可定=df_drug_resuvatin_before[df_drug_resuvatin_before['resuvatin_brand']=='可定']

# print(df_drug_resuvatin_before_京新.患者编号.nunique())
# print(df_drug_resuvatin_before_可定.患者编号.nunique())

# df_drug_resuvatin_after_京新=df_drug_resuvatin_after[df_drug_resuvatin_after['resuvatin_brand']=='京新']
# df_drug_resuvatin_after_可定=df_drug_resuvatin_after[df_drug_resuvatin_after['resuvatin_brand']=='可定']

# print(df_drug_resuvatin_after_京新.患者编号.nunique())
# print(df_drug_resuvatin_after_可定.患者编号.nunique())

## 其他他汀数据处理

In [96]:
df_drug_vatin_other=pd.read_csv(project_path + '/data/processed_data/df_2.3.4_其它他汀用药.csv')
if 'Unnamed: 0' in df_drug_vatin_other.columns:
    del df_drug_vatin_other['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
df_drug_vatin_other.shape

(275612, 13)

In [98]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期', '开药总剂量'],
      dtype='object')

In [99]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长'],
      dtype='object')

In [100]:
# 打标签
df_drug_vatin_other['resuvatin_brand']='其他汀'

### 删除重复

In [101]:
df_drug_vatin_other.shape

(275612, 14)

In [102]:
# 同一天多次用药记录，取最后一次;
df_drug_vatin_other['医嘱开始日期'] = df_drug_vatin_other.医嘱开始时间.apply(lambda x: str(x).split(' ')[0])
df_drug_vatin_other = df_drug_vatin_other.sort_values(['患者编号','药物通用名称','医嘱开始时间'],ascending=True)
df_drug_vatin_other = df_drug_vatin_other.drop_duplicates(subset=['患者编号','药物通用名称','医嘱开始日期'],keep='last')

In [103]:
df_drug_vatin_other.shape

(274168, 15)

In [104]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '医嘱开始日期'],
      dtype='object')

### 计算日剂量

#### 转换频率

In [105]:
# 清洗频次
half=['QOD(8am)','QOD5 （8pm）','QOD1(12N)','QOD4 (4pm)','QON （9pm）']
one=['QD(8am)','01 每日一次','QN每晚 (8pm)','ONCE','1/晚(10PM)','05 每晚一次','QD1(8am)','QD7(1pm)','QD6(4pm)','QD4 (5pm)','QD3(12N)',
     'QD12(10am)']
two=['BID(8-4)','02 每日两次','BID1 (8-4)口服','Q12H (8-8)']
three=['TID(8-12-4)','03 每日三次']

df_drug_vatin_other['频率名称']=df_drug_vatin_other['频率名称'].apply(lambda x: 0.5 if x in half else
                                                                          1 if x in one else
                                                                          2 if x in two else
                                                                          3 if x in three else np.nan)

In [106]:
df_drug_vatin_other.频率名称.value_counts()

1.0    265641
2.0      7591
3.0       870
0.5        61
Name: 频率名称, dtype: int64

In [107]:
# 删除频率为空的用药
df_drug_vatin_other = df_drug_vatin_other[df_drug_vatin_other.频率名称.notnull()]

In [108]:
df_drug_vatin_other.shape

(274163, 15)

In [109]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.2.1_其他汀_转换频率.csv')

#### 单次剂量处理

In [110]:
df_drug_vatin_other.药物剂量单位名称.value_counts()

mg    163605
片      42642
盒        131
胶囊        22
g          2
Name: 药物剂量单位名称, dtype: int64

In [111]:
# 预处理片、盒、胶囊等单位
df_drug_vatin_other= df_drug_vatin_other.reset_index(drop=True)
for i in range(df_drug_vatin_other.shape[0]):
    u = df_drug_vatin_other.loc[i,'药物剂量单位名称']
    if pd.isnull(u):
        continue
    # 剂量单位为g
    if re.match('g',u):
        if re.match('阿托伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
             df_drug_vatin_other.loc[i,'药物剂量']=10
    # 剂量单位为盒
    elif re.match('盒',u):
        if re.match('阿托伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            if df_drug_vatin_other.loc[i,'药物剂量']>=40:
                continue
            else:
                df_drug_vatin_other.loc[i,'药物剂量']=20
        elif re.match('匹伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            df_drug_vatin_other.loc[i,'药物剂量']=2
    # 剂量单位为胶囊
    elif re.match('胶囊',u):
        if re.match('氨氯地平阿托伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            df_drug_vatin_other.loc[i,'药物剂量']=10
        elif re.match('氟伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            df_drug_vatin_other.loc[i,'药物剂量']=40
    # 剂量单位为片
    elif re.match('片',u):
#         break
        if re.match('阿托伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            # 正则
            if re.match('.*20.*', df_drug_vatin_other.loc[i,'药物规格']):
                df_drug_vatin_other.loc[i,'药物剂量']=20*df_drug_vatin_other.loc[i,'药物剂量']
            else:
                df_drug_vatin_other.loc[i,'药物剂量']=10*df_drug_vatin_other.loc[i,'药物剂量']
        elif re.match('氨氯地平阿托伐他汀钙',df_drug_vatin_other.loc[i,'药物通用名称']):
            if df_drug_vatin_other.loc[i,'药物剂量']>=2:
                df_drug_vatin_other.loc[i,'药物剂量']=5
            else:
                df_drug_vatin_other.loc[i,'药物剂量']=5*df_drug_vatin_other.loc[i,'药物剂量']
        elif re.match('洛伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            df_drug_vatin_other.loc[i,'药物剂量']=20
        elif re.match('匹伐他汀钙',df_drug_vatin_other.loc[i,'药物通用名称']):
            if df_drug_vatin_other.loc[i,'药物剂量']>=2:
                df_drug_vatin_other.loc[i,'药物剂量']=2
            else:
                df_drug_vatin_other.loc[i,'药物剂量']=2*df_drug_vatin_other.loc[i,'药物剂量']
#                 break
        elif re.match('辛伐他汀',df_drug_vatin_other.loc[i,'药物通用名称']):
            if df_drug_vatin_other.loc[i,'药物剂量']>=5:
                df_drug_vatin_other.loc[i,'药物剂量']=40
            else:
                df_drug_vatin_other.loc[i,'药物剂量']=40*df_drug_vatin_other.loc[i,'药物剂量']    

In [117]:
# 调整单次剂量的数值
df_drug_vatin_other= df_drug_vatin_other.reset_index(drop=True)
for i in range(df_drug_vatin_other.shape[0]):
    n = df_drug_vatin_other.loc[i,'药物通用名称']
    # 阿托伐他汀
    if re.match('阿托伐他汀|阿乐片|立普妥', n):
        if df_drug_vatin_other.loc[i,'药物剂量']<=4:
            df_drug_vatin_other.loc[i,'药物剂量']=20*df_drug_vatin_other.loc[i,'药物剂量']
        elif df_drug_vatin_other.loc[i,'药物剂量']>80:
            df_drug_vatin_other.loc[i,'药物剂量']=20
    # 氨氯地平阿托伐他汀
    if re.match('氨氯地平阿托伐他汀钙', n):
        if df_drug_vatin_other.loc[i,'药物剂量']<=2:
            df_drug_vatin_other.loc[i,'药物剂量']=5*df_drug_vatin_other.loc[i,'药物剂量']
        elif df_drug_vatin_other.loc[i,'药物剂量']>10:
            df_drug_vatin_other.loc[i,'药物剂量']=10
    # 匹伐他汀钙
    if re.match('匹伐他汀', n):
        if df_drug_vatin_other.loc[i,'药物剂量']<=0.5:
            df_drug_vatin_other.loc[i,'药物剂量']=2*df_drug_vatin_other.loc[i,'药物剂量']
        elif df_drug_vatin_other.loc[i,'药物剂量']>4:
            df_drug_vatin_other.loc[i,'药物剂量']=2
    # 辛伐他汀
    if re.match('辛伐他汀', n):
        if df_drug_vatin_other.loc[i,'药物剂量']>80:
            df_drug_vatin_other.loc[i,'药物剂量']=40

In [118]:
df_drug_vatin_other.shape

(274163, 15)

In [119]:
df_drug_vatin_other.药物剂量.value_counts()

20.00     115267
2.00       36818
40.00      33437
10.00      30903
5.00       18479
4.00       16028
30.00       6881
7.50        3553
15.00       3063
80.00       2598
3.00        2407
60.00       1513
1.00        1192
50.00       1145
25.00        523
45.00        131
70.00         75
12.00         26
22.00         25
75.00         24
11.00         17
35.00         12
5.50          10
2.50           9
100.00         8
14.00          7
41.00          4
3.75           2
28.00          2
21.00          1
7.00           1
48.00          1
1.50           1
Name: 药物剂量, dtype: int64

In [120]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.2.2_单次剂量处理.csv')

#### 计算日剂量

In [121]:
df_drug_vatin_other['日剂量']=df_drug_vatin_other['药物剂量'].astype('float') * df_drug_vatin_other['频率名称'].astype('float')

In [124]:
# 调整异常值
df_drug_vatin_other= df_drug_vatin_other.reset_index(drop=True)
for i in range(df_drug_vatin_other.shape[0]):
    n = df_drug_vatin_other.loc[i,'药物通用名称']
    # 阿托伐他汀
    if re.match('阿托伐他汀|阿乐片|立普妥', n):
        if df_drug_vatin_other.loc[i,'日剂量']>80:
            df_drug_vatin_other.loc[i,'日剂量']=80
    # 氟伐他汀
    if re.match('氟伐他汀', n):
        if df_drug_vatin_other.loc[i,'日剂量']>100:
            df_drug_vatin_other.loc[i,'日剂量']=100
    # 辛伐他汀
    if re.match('辛伐他汀', n):
        if df_drug_vatin_other.loc[i,'日剂量']>80:
            df_drug_vatin_other.loc[i,'日剂量']=80

In [ ]:
df_drug_vatin_other.shape

In [127]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.2.3_计算其他汀日剂量.csv')

### 计算他汀药物时长和开药总剂量

#### 计算世纪坛--用药时长

In [130]:
# 对于世纪坛医院用药结束时间为空的数据，将下一次开药时间作为其上一次结束时间。
temp_list=[]
for i in df_drug_vatin_other.患者编号.unique():
    temp = df_drug_vatin_other[df_drug_vatin_other['患者编号']==i]
    temp = temp.sort_values(by=['医嘱开始时间'],ascending=True).reset_index(drop=True)
    hospital = temp.loc[0,'data_source']
    if re.match('世纪坛医院',hospital) and (temp.shape[0]>1):
        for j in range(temp.shape[0]):
            # 医嘱结束时间为空
            if pd.isnull(temp.loc[j,'医嘱停止时间']) and (j+1<temp.shape[0]):
                # 提取下一次用药记录
                temp_end = temp[temp['医嘱开始日期'] > temp.loc[j,'医嘱开始日期']].reset_index(drop=True)
                if temp_end.shape[0]>0:
                    temp.loc[j,'医嘱停止时间'] = temp_end.loc[0,'医嘱开始时间']
    temp_list.append(temp)

In [131]:
df_drug_vatin_other=pd.concat(temp_list,axis=0)
del temp_list

In [132]:
df_drug_vatin_other.shape

(274163, 16)

In [134]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '医嘱开始日期', '日剂量'],
      dtype='object')

In [136]:
# 用药时长=医嘱结束时间-医嘱开始时间+1
df_drug_vatin_other['用药时长'] = df_drug_vatin_other.apply(lambda x: (pd.to_datetime(x['医嘱停止时间'])-pd.to_datetime(x['医嘱开始时间'])).days + 1 if x['data_source']=='世纪坛医院' else np.nan,axis=1)

In [137]:
df_drug_vatin_other.shape

(274163, 16)

In [140]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.3.1_计算他汀药物_世纪坛_用药时长.csv')

#### 计算开药总剂量

In [144]:
# 盒--280mg；片--10mg
df_drug_vatin_other = df_drug_vatin_other.reset_index(drop=True)
for i in range(df_drug_vatin_other.shape[0]):
    prescription_u = df_drug_vatin_other.loc[i,'开药总数量单位名称']
    # 世纪坛医院：日剂量*用药时长
    if pd.isnull(prescription_u):
        df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'用药时长'] * df_drug_vatin_other.loc[i,'日剂量']
    # 人民医院: 盒
    elif re.match('盒', prescription_u):
        name = df_drug_vatin_other.loc[i,'药物通用名称']
        # 阿托伐他汀
        if re.match('阿托伐他汀', name):
            # 正则
            if re.match('.*20.*', df_drug_vatin_other.loc[i,'药物规格']):
                df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*140
            else:
                df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*70
        # 氨氯地平阿托伐他汀钙
        elif re.match('氨氯地平', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*35
        # 氟伐他汀
        elif re.match('氟伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*280
        # 洛伐他汀
        elif re.match('洛伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*200
        # 匹伐他汀钙
        elif re.match('匹伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*14
        # 辛伐他汀
        elif re.match('辛伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*200
    # 人民医院: 胶囊
    elif re.match('胶囊', prescription_u):
        df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*40
    # 人民医院：片
    elif re.match('片', prescription_u):
        name = df_drug_vatin_other.loc[i,'药物通用名称']
        # 阿托伐他汀
        if re.match('阿托伐他汀', name):
            # 正则
            if re.match('.*20.*', df_drug_vatin_other.loc[i,'药物规格']):
                df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*20
            else:
                df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*10
        # 氨氯地平阿托伐他汀钙
        if re.match('氨氯地平', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*5
        # 洛伐他汀
        if re.match('洛伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*20
        # 匹伐他汀钙
        if re.match('匹伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*2
        # 辛伐他汀
        if re.match('辛伐他汀', name):
            df_drug_vatin_other.loc[i,'开药总剂量']= df_drug_vatin_other.loc[i,'开药总数量']*40

In [145]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.3.2_计算其他汀开药总剂量.csv')

#### 计算人民医院医嘱结束时间

In [149]:
df_drug_vatin_other.shape

(274163, 17)

In [153]:
# 计算用药时长
df_drug_vatin_other['用药时长']= df_drug_vatin_other.apply(lambda x:round(x['开药总剂量']/x['日剂量'],0) if x['data_source']=='人民医院' else x['用药时长'],axis=1)

In [155]:
df_drug_vatin_other['医嘱停止时间']=df_drug_vatin_other.apply(lambda x: pd.to_datetime(x['医嘱开始时间'])+datetime.timedelta(days=x['用药时长'])if (x['data_source']=='人民医院') & (pd.notnull(x['用药时长'])) 
                                                                    else x['医嘱停止时间'],axis=1)

In [157]:
df_drug_vatin_other.shape

(274163, 17)

In [156]:
df_drug_vatin_other.to_csv(project_path+'/data/processed_data/政策性/df_3.1.3.3_计算其他汀_医嘱结束时间.csv')

In [279]:
df_drug_vatin_other=pd.read_csv(project_path+'/data/processed_data/政策性/df_3.1.3.3_计算其他汀_医嘱结束时间.csv')
if 'Unnamed: 0' in df_drug_vatin_other.columns:
    del df_drug_vatin_other['Unnamed: 0']

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [280]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '医嘱开始日期', '日剂量', '开药总剂量'],
      dtype='object')

## 合并瑞舒他汀和其他他汀用药

In [281]:
df_drug_resuvatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '日剂量', '医嘱开始日期', '开药总剂量'],
      dtype='object')

In [282]:
df_drug_vatin_other.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '医嘱开始日期', '日剂量', '开药总剂量'],
      dtype='object')

In [283]:
# 调整顺序
df_drug_resuvatin = df_drug_resuvatin[['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
                                       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
                                       'resuvatin_brand', '医嘱开始日期', '日剂量', '开药总剂量']]

In [684]:
df_drug_vatin=pd.concat([df_drug_resuvatin,df_drug_vatin_other],axis=0).sort_values(['患者编号','医嘱开始时间'],ascending=True).reset_index(drop=True)

In [685]:
df_drug_vatin.shape

(328634, 17)

### 拆分出瑞舒伐他汀和其他他汀

In [686]:
df_drug_vatin.药物通用名称.unique()

array(['阿乐片(ZC/10mg/片)', '阿托伐他汀钙片(10mg/片(ZX))', '匹伐他汀钙片(ZD/ 2mg/片)',
       '立普妥片(ZC/ 20mg/片)', '阿托伐他汀钙片(20mg/片(ZX))', '瑞舒伐他汀钙片(10mg/片(ZX))',
       '阿托伐他汀钙片(20mg/片)', '可定片(ZDL/ 10mg/片)', '瑞舒伐他汀钙片(ZD/ 10mg/片)',
       '瑞舒伐他汀钙片(10mg/片)', '氟伐他汀钠缓释片(ZD/ 80mg/片)',
       '氨氯地平阿托伐他汀钙片(ZDL/ 5mg/10mg/片)', '匹伐他汀钙', '阿托伐他汀钙', '瑞舒伐他汀钙',
       '氨氯地平阿托伐他汀钙', '辛伐他汀', '氟伐他汀', '阿托伐他汀钙片', '洛伐他汀', '瑞舒伐他汀钙片'],
      dtype=object)

In [687]:
df_drug_vatin['瑞舒伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('瑞舒伐他汀|可定片',x) else 0)
df_drug_vatin['阿托伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('阿托伐他汀|阿乐片|立普妥片',x) else 0)
df_drug_vatin['匹伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('匹伐他汀',x) else 0)
df_drug_vatin['氟伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('氟伐他汀',x) else 0)
df_drug_vatin['氨氯地平阿托伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('氨氯地平阿托伐他汀',x) else 0)
df_drug_vatin['辛伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('辛伐他汀',x) else 0)
df_drug_vatin['洛伐他汀']=df_drug_vatin.药物通用名称.apply(lambda x: 1 if re.match('洛伐他汀',x) else 0)

In [688]:
df_drug_vatin.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       'resuvatin_brand', '医嘱开始日期', '日剂量', '开药总剂量', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀',
       '氟伐他汀', '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀'],
      dtype='object')

In [689]:
# 调整顺序
df_drug_vatin=df_drug_vatin[['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长',
       '医嘱开始日期', '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀',
       '氟伐他汀', '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀']]

### 删除同一天的联合用药

In [690]:
# 如果同一天存在多条用药，以最后一条为准
df_drug_vatin=df_drug_vatin.sort_values(['患者编号','医嘱开始时间']).drop_duplicates(['患者编号','医嘱开始日期'],keep='last').reset_index(drop=True)

In [691]:
df_drug_vatin.shape

(327565, 24)

In [692]:
df_drug_vatin.to_csv(project_path+'/data/processed_data/政策性/df_3.2_合并他汀类药物.csv')

## 纳入高血脂、冠心病患者

In [693]:
# 高脂血症、冠心病
df_diagnose_disease = df_diagnose[df_diagnose['诊断名称'].str.contains('高脂血症|冠心病|冠状动脉粥样硬化性心脏病')]
# 删除重复诊断
df_diagnose_disease = df_diagnose_disease.drop_duplicates(['患者编号','诊断名称'],keep='first')

In [694]:
print(df_diagnose_disease.shape)
print(df_diagnose_disease.患者编号.nunique())

(18322, 5)
9598


In [695]:
df_diagnose_disease.head(1)

,患者编号,诊断名称,诊断日期,是否主要诊断,诊断类别名称
2,pat_05eab24ef6eb4e0827cd26e1553a0f64,动脉粥样硬化并高脂血症,2018-02-06 13:20:56,是,门诊诊断


In [696]:
df_diagnose_disease.to_csv(project_path+'/data/processed_data/df_temp_冠心病和高脂血病诊断.csv')

In [697]:
df_drug_resuvatin.shape

(54471, 17)

In [698]:
# 纳入高脂血症、冠心病
df_drug_vatin_disease = df_drug_vatin[df_drug_vatin.患者编号.isin(df_diagnose_disease.患者编号.unique())]
# 删除重复数据
df_drug_vatin_disease = df_drug_vatin_disease.drop_duplicates(df_drug_vatin_disease.columns,keep='first')

In [699]:
print(df_drug_vatin_disease.shape)
print(df_drug_vatin_disease.患者编号.nunique())

(66787, 24)
9598


In [700]:
df_drug_vatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,日剂量,开药总剂量,resuvatin_brand,瑞舒伐他汀,阿托伐他汀,匹伐他汀,氟伐他汀,氨氯地平阿托伐他汀,辛伐他汀,洛伐他汀
35,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,10.0,50.0,京新,1,0,0,0,0,0,0


### 冠心病标签

In [701]:
# 提取冠心病
df_disease_CHD = df_diagnose_disease[df_diagnose_disease.诊断名称.str.contains('冠心病|冠状动脉粥样硬化性心脏病')]
df_drug_vatin_disease = pd.merge(df_drug_vatin_disease, df_disease_CHD[['患者编号','诊断名称']], how='left', on='患者编号')

In [702]:
# 打冠心病标签
df_drug_vatin_disease = df_drug_vatin_disease.rename(columns={'诊断名称':'冠心病'})
df_drug_vatin_disease['冠心病'] = df_drug_vatin_disease['冠心病'].apply(lambda x: 0 if pd.isnull(x) else 1)
# 删除相同诊断造成的重复
df_drug_vatin_disease =df_drug_vatin_disease.drop_duplicates(df_drug_vatin_disease.columns,keep='first')

In [703]:
df_drug_vatin_disease.冠心病.value_counts()

1    43088
0    23699
Name: 冠心病, dtype: int64

In [704]:
df_drug_vatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,开药总剂量,resuvatin_brand,瑞舒伐他汀,阿托伐他汀,匹伐他汀,氟伐他汀,氨氯地平阿托伐他汀,辛伐他汀,洛伐他汀,冠心病
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,50.0,京新,1,0,0,0,0,0,0,1


### 高脂血病标签

In [705]:
# 打高脂血症标签
df_disease_HLP = df_diagnose_disease[df_diagnose_disease.诊断名称.str.contains('高脂血症')]
df_drug_vatin_disease = pd.merge(df_drug_vatin_disease, df_disease_HLP[['患者编号','诊断名称']], how='left', on='患者编号') 

In [706]:
df_drug_vatin_disease = df_drug_vatin_disease.rename(columns={'诊断名称':'高脂血症'})
df_drug_vatin_disease['高脂血症'] = df_drug_vatin_disease['高脂血症'].apply(lambda x: 0 if pd.isnull(x) else 1)
# 删除相同诊断造成的重复
df_drug_vatin_disease =df_drug_vatin_disease.drop_duplicates(df_drug_vatin_disease.columns,keep='first')

In [707]:
df_drug_vatin_disease.高脂血症.value_counts()

1    65442
0     1345
Name: 高脂血症, dtype: int64

In [708]:
df_drug_vatin_disease.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,resuvatin_brand,瑞舒伐他汀,阿托伐他汀,匹伐他汀,氟伐他汀,氨氯地平阿托伐他汀,辛伐他汀,洛伐他汀,冠心病,高脂血症
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,京新,1,0,0,0,0,0,0,1,1


### 冠心病&高脂血病-标签

In [709]:
# 同时发生冠心病和高血脂病
df_drug_vatin_disease['冠心病_高脂血症']=df_drug_vatin_disease.apply(lambda x: 1 if (x['冠心病']==1) & (x['高脂血症']==1) 
                                                                      else 0,axis=1)

In [710]:
df_drug_vatin_disease.shape

(66787, 27)

In [711]:
df_drug_vatin_disease.to_csv(project_path+'/data/processed_data/政策性/df_3.3.3_打冠心病和高脂血病标签.csv')

## 排除癌症患者

In [712]:
# 癌症患者
df_diagnose_cancer = df_diagnose[df_diagnose['诊断名称'].str.contains('癌|恶性肿瘤')]
# 排除癌症患者
df_drug_vatin_cancer = df_drug_vatin_disease[~ df_drug_vatin_disease.患者编号.isin(df_diagnose_cancer.患者编号.unique())]

In [713]:
df_drug_vatin_cancer.shape

(65898, 27)

In [714]:
df_drug_vatin_cancer.to_csv(project_path+'/data/processed_data/政策性/df_3.4_排除癌症患者.csv')

In [715]:
df_drug_vatin_cancer=pd.read_csv(project_path+'/data/processed_data/政策性/df_3.4_排除癌症患者.csv')
if 'Unnamed: 0' in df_drug_vatin_cancer.columns:
    del df_drug_vatin_cancer['Unnamed: 0']

## 人口学特征

In [213]:
'''
AGE=患者初次处方京新和/或可定日期-患者的出生日期
'''

'\nAGE=患者初次处方京新和/或可定日期-患者的出生日期\n'

### 人民医院--人口学数据

In [95]:
df_popu_1 = pd.read_excel(r'./data/raw_data/人民医院/1-患者基本信息.xlsx')

In [96]:
df_popu_1.shape

(44998, 3)

In [97]:
df_popu_1.columns

Index(['患者编号', '性别', '出生日期'], dtype='object')

### 世纪坛医院--人口学数据

In [98]:
# 世纪坛医院--人口学数据
df_popu_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                              sheet_name='患者')

KeyboardInterrupt: 

In [ ]:
df_popu_2.shape

In [ ]:
df_popu_2.columns

In [ ]:
df_popu_2 = df_popu_2[['patient_SN', '性别', '出生日期']]
df_popu_2.rename(columns={'patient_SN':'患者编号'},inplace=True)

### 合并人口学特征

In [ ]:
df_popu = pd.concat([df_popu_1,df_popu_2],axis=0).reset_index(drop=True)

In [ ]:
df_popu.shape

In [ ]:
# 删除重复数据
df_popu = df_popu.drop_duplicates(df_popu.columns, keep='first')

In [ ]:
df_popu.shape

In [ ]:
df_popu.head()

In [ ]:
df_popu.to_csv(project_path+'/data/processed_data/df_3.5.3_合并人口学特征.csv')

In [716]:
df_popu=pd.read_csv(project_path+'/data/processed_data/df_3.5.3_合并人口学特征.csv')
if 'Unnamed: 0' in df_popu.columns:
    del df_popu['Unnamed: 0']

In [717]:
df_popu.columns

Index(['患者编号', '性别', '出生日期'], dtype='object')

### 并入人口学特征

In [718]:
# 性别：男-1；女-0
df_popu['性别'] = df_popu.性别.apply(lambda x: 1 if re.match('男',x) else 0)

In [719]:
df_popu.性别.value_counts()

1    34957
0    30847
Name: 性别, dtype: int64

In [720]:
# 提取出生年
df_popu['出生日期'] = df_popu.出生日期.apply(lambda x: np.nan if pd.isnull(x) else
                                     str(x).split('-')[0] if '-' in str(x) else x)

In [721]:
df_popu.head()

,患者编号,性别,出生日期
0,pat_0d9297a346707bce01f0b4637484efe9,0,1956.0
1,pat_0ebd83a8c3990a103016a650f84616f8,0,1962.0
2,pat_13da443c402ad02c9e9de0c5c0751eda,0,1973.0
3,pat_2c40bef1b35d17984b1d393c6be55688,0,1952.0
4,pat_0f6bf632666460422c4923f591ede61f,0,1943.0


In [722]:
df_drug_vatin_cancer.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症'],
      dtype='object')

In [723]:
# 计算年龄
# 初次用药年
df_drug_vatin_popu = df_drug_vatin_cancer.copy().sort_values(['患者编号','医嘱开始时间'])[['患者编号','医嘱开始时间']].reset_index(drop=True)
df_drug_vatin_popu = df_drug_vatin_popu.drop_duplicates(['患者编号'],keep='first').rename(columns={'医嘱开始时间':'初次用药年'})
df_drug_vatin_popu['初次用药年']=df_drug_vatin_popu.初次用药年.apply(lambda x: str(x).split('-')[0])

In [724]:
# 计算年龄
df_drug_vatin_popu = df_drug_vatin_cancer.copy().reset_index(drop=True)
df_drug_start_time = df_drug_vatin_popu.sort_values(['患者编号','医嘱开始时间']).drop_duplicates(['患者编号'],keep='first')
df_drug_start_time = df_drug_start_time[['患者编号','医嘱开始时间']].rename(columns={'医嘱开始时间':'初次用药年'})
df_drug_start_time['初次用药年'] =df_drug_start_time.初次用药年.apply(lambda x: str(x).split('-')[0])
# 并入到用药数据
df_drug_vatin_popu = pd.merge(df_drug_vatin_popu,df_drug_start_time,on='患者编号',how='left')
# 并入人口学特征
df_drug_vatin_popu = pd.merge(df_drug_vatin_popu,df_popu,on='患者编号',how='left')
# 计算年龄
df_drug_vatin_popu['年龄'] = df_drug_vatin_popu.apply(lambda x:  np.nan if pd.isnull(x['出生日期']) else
                                                                        float(x['初次用药年'])-float(x['出生日期']) ,axis=1)

In [725]:
df_drug_vatin_popu.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄'],
      dtype='object')

In [726]:
df_drug_vatin_popu.head(2)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,氨氯地平阿托伐他汀,辛伐他汀,洛伐他汀,冠心病,高脂血症,冠心病_高脂血症,初次用药年,性别,出生日期,年龄
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,0,0,0,1,1,1,2019,0,1952,67.0
1,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-16 00:00:00,2019-05-13 00:00:00,瑞舒伐他汀钙片(10mg/片(ZX))10mg*28片(ZX),20.0,NaN,1.0,NaN,...,0,0,0,1,1,1,2019,0,1952,67.0


In [727]:
df_drug_vatin_popu.to_csv(project_path+'/data/processed_data/政策性/df_3.5.4_并入人口学特征.csv')

## 个人史既往史

### 人民医院--既往史

In [230]:
df_history_1 = pd.read_excel(r'./data/raw_data/人民医院/5-病史（1诉5史）.xlsx')

KeyboardInterrupt: 

In [167]:
# 删除重复值
df_history_1 = df_history_1.drop_duplicates(['患者编号','主诉'],keep='first')[['患者编号','主诉']]
# 删除空值
df_history_1 = df_history_1[df_history_1.主诉.notnull()]

In [168]:
df_history_1.shape

(83658, 2)

In [169]:
df_history_1.head(1)

,患者编号,主诉
0,pat_0d9297a346707bce01f0b4637484efe9,高血压平时在积水潭医院诊疗


In [170]:
df_history_1['吸烟史']=df_history_1.主诉.apply(lambda x: 0 if pd.isnull(x) else
                                                      1 if re.match('.*[^无未否戒]*烟',x) else 0)
df_history_1['饮酒史']=df_history_1.主诉.apply(lambda x: 0 if pd.isnull(x) else
                                                      1 if re.match('.*[^无未否戒]*酒[^精]',x) else 0)

In [171]:
# groupby
df_history_1_smoke=df_history_1.groupby(['患者编号'])['吸烟史'].sum().to_frame().reset_index()
df_history_1_smoke['吸烟史']=df_history_1_smoke.吸烟史.apply(lambda x: np.nan if pd.isnull(x) else 
                                                                    1 if x>=1 else 0)
df_history_1_drink=df_history_1.groupby(['患者编号'])['饮酒史'].sum().to_frame().reset_index()
df_history_1_drink['饮酒史']=df_history_1_drink.饮酒史.apply(lambda x: np.nan if pd.isnull(x) else 
                                                                    1 if x>=1 else 0)

In [172]:
# smoke+drink
df_history_1=pd.merge(df_history_1_smoke,df_history_1_drink,on='患者编号',how='inner')

In [173]:
df_history_1.shape

(36630, 3)

In [174]:
df_history_1.to_csv(project_path+'/data/processed_data/df_3.5.1_人民医院_既往史.csv')

In [728]:
df_history_1=pd.read_csv(project_path+'/data/processed_data/df_3.5.1_人民医院_既往史.csv')
if 'Unnamed: 0' in df_history_1.columns:
    del df_history_1['Unnamed: 0']

In [729]:
df_history_1.columns

Index(['患者编号', '吸烟史', '饮酒史'], dtype='object')

### 提取高血压糖尿病

In [730]:
# 提取高血压
df_diagnose_EH = df_diagnose[df_diagnose.诊断名称.str.contains('高血压')].drop_duplicates(['患者编号'],keep='first')[['患者编号']]
df_diagnose_EH['高血压'] = 1
# 提取糖尿病
df_diagnose_DM = df_diagnose[df_diagnose.诊断名称.str.contains('糖尿病')].drop_duplicates(['患者编号'],keep='first')[['患者编号']]
df_diagnose_DM['糖尿病'] = 1

In [731]:
print(df_diagnose_EH.shape)
print(df_diagnose_DM.shape)

(7277, 2)
(2662, 2)


In [732]:
df_diagnose_EH.head(1)

,患者编号,高血压
491,pat_026a30f807b837014dd9f700c0b93913,1


### 并入个人史既往史

In [733]:
# 并入个人史
df_drug_vatin_popu = pd.merge(df_drug_vatin_popu,df_history_1,on='患者编号',how='left')
# 并入高血压
df_drug_vatin_popu = pd.merge(df_drug_vatin_popu,df_diagnose_EH,on='患者编号',how='left')
# 并入糖尿病
df_drug_vatin_popu = pd.merge(df_drug_vatin_popu,df_diagnose_DM,on='患者编号',how='left')

In [734]:
df_drug_vatin_popu['吸烟史']=df_drug_vatin_popu.吸烟史.fillna(0)
df_drug_vatin_popu['饮酒史']=df_drug_vatin_popu.饮酒史.fillna(0)
df_drug_vatin_popu['高血压']=df_drug_vatin_popu.高血压.fillna(0)
df_drug_vatin_popu['糖尿病']=df_drug_vatin_popu.糖尿病.fillna(0)

In [735]:
df_drug_vatin_popu.head(1)

,data_source,患者编号,药物通用名称,医嘱开始时间,医嘱停止时间,药物规格,药物剂量,药物剂量单位名称,频率名称,开药总数量,...,高脂血症,冠心病_高脂血症,初次用药年,性别,出生日期,年龄,吸烟史,饮酒史,高血压,糖尿病
0,世纪坛医院,0020640c4052f1dae97fcf39be390e1b,瑞舒伐他汀钙片(10mg/片(ZX)),2019-04-04 15:35:28,2019-04-09 07:42:00,10mg*28片(ZX),10.0,NaN,1.0,NaN,...,1,1,2019,0,1952,67.0,0.0,0.0,1.0,0.0


In [736]:
df_drug_vatin_popu.to_csv(project_path+'/data/processed_data/政策性/df_3.6.3_并入个人史既往史.csv')

## 计算换药率

In [737]:
'''
2018.03.23-2019.03.22政策前
2019.03.23-2020.03.22政策后
    分组group：
    换药率：未换药、换药一次、换药两次
    初始治疗选择
'''

'\n2018.03.23-2019.03.22政策前\n2019.03.23-2020.03.22政策后\n    分组group：\n    换药率：未换药、换药一次、换药两次\n    初始治疗选择\n'

In [738]:
df_drug_vatin_policy = df_drug_vatin_popu.copy().reset_index(drop=True)

In [739]:
df_drug_vatin_policy.shape

(65898, 35)

### 限定政策前后时间范围

In [740]:
df_drug_vatin_policy = df_drug_vatin_policy[(pd.to_datetime(df_drug_vatin_policy['医嘱开始时间'])<=pd.to_datetime('2020-03-23')) &
                                           (pd.to_datetime(df_drug_vatin_policy['医嘱开始时间'])>=pd.to_datetime('2018-03-23')) ]

In [741]:
df_drug_vatin_policy.shape

(58301, 35)

### 患者分组

In [742]:
'''
患者分为三组：①用药疗程在政策执行之前；②用药疗程跨越政策前后；③用药疗程在政策实施之后
'''

'\n患者分为三组：①用药疗程在政策执行之前；②用药疗程跨越政策前后；③用药疗程在政策实施之后\n'

In [743]:
df_drug_vatin_policy.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病'],
      dtype='object')

In [744]:
# 提取首次用药时间和末次用药时间
df_drug_vatin_policy=df_drug_vatin_policy.sort_values(['患者编号','医嘱开始时间'],ascending=True).reset_index(drop=True)
df_drug_first_time=df_drug_vatin_policy.drop_duplicates(['患者编号'],keep='first')[['患者编号','医嘱开始时间']].rename(columns={'医嘱开始时间':'首次用药时间'})
df_drug_last_time=df_drug_vatin_policy.drop_duplicates(['患者编号'],keep='last')[['患者编号','医嘱开始时间']].rename(columns={'医嘱开始时间':'末次用药时间'})
# 提取日期
df_drug_first_time['首次用药时间']=df_drug_first_time['首次用药时间'].apply(lambda x: str(x).split(' ')[0])
df_drug_last_time['末次用药时间']=df_drug_last_time['末次用药时间'].apply(lambda x: str(x).split(' ')[0])

In [745]:
# 并入首次用药时间和末次用药时间
df_drug_vatin_policy=pd.merge(df_drug_vatin_policy,df_drug_first_time,on='患者编号',how='left')
df_drug_vatin_policy=pd.merge(df_drug_vatin_policy,df_drug_last_time,on='患者编号',how='left')

In [746]:
df_drug_vatin_policy.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间'],
      dtype='object')

In [747]:
# 分组
df_drug_vatin_policy['group']=df_drug_vatin_policy.apply(lambda x: 1 if (pd.to_datetime(x['首次用药时间'])<=pd.to_datetime('2019-03-22')) & (pd.to_datetime(x['末次用药时间'])<=pd.to_datetime('2019-03-22')) else
                                                        2 if (pd.to_datetime(x['首次用药时间'])<=pd.to_datetime('2019-03-22')) & (pd.to_datetime(x['末次用药时间'])>=pd.to_datetime('2019-03-23')) else
                                                        3 if (pd.to_datetime(x['首次用药时间'])>=pd.to_datetime('2019-03-23')) & (pd.to_datetime(x['末次用药时间'])>=pd.to_datetime('2019-03-23')) else 4, axis=1)

In [748]:
df_drug_vatin_policy.group.value_counts()

2    47065
1     6888
3     4348
Name: group, dtype: int64

In [749]:
df_drug_vatin_policy.to_csv(project_path+'/data/processed_data/政策性/df_3.7.2_患者分组.csv')

### 划分政策前后观察期

In [863]:
df_drug_vatin_policy.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group'],
      dtype='object')

In [864]:
# 政策前1年观察期和政策后1年观察期
df_drug_vatin_before=df_drug_vatin_policy[(pd.to_datetime(df_drug_vatin_policy['医嘱开始日期'])<=pd.to_datetime('2019-03-22')) &
                                                 (pd.to_datetime(df_drug_vatin_policy['医嘱开始日期'])>=pd.to_datetime('2018-03-23'))]
df_drug_vatin_after=df_drug_vatin_policy[(pd.to_datetime(df_drug_vatin_policy['医嘱开始日期'])<=pd.to_datetime('2020-03-22')) &
                                                 (pd.to_datetime(df_drug_vatin_policy['医嘱开始日期'])>=pd.to_datetime('2019-03-23'))]

In [865]:
print(df_drug_vatin_before.shape)
print(df_drug_vatin_after.shape)

(32058, 38)
(26243, 38)


In [866]:
print(df_drug_vatin_before.group.value_counts())
print(df_drug_vatin_after.group.value_counts())

2    25170
1     6888
Name: group, dtype: int64
2    21895
3     4348
Name: group, dtype: int64


In [867]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.3.1_政策前_他汀数据.csv')
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.3.2_政策后_他汀数据.csv')

### 换药次数

In [868]:
df_drug_vatin_before.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group'],
      dtype='object')

#### 政策前

In [869]:
# 计算换药次数
temp_list=[]
for i in df_drug_vatin_before.患者编号.unique():
    temp=df_drug_vatin_before[df_drug_vatin_before['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    # 统计换药次数
    count=0
    brand=temp.loc[0,'resuvatin_brand']
    for j in range(1,temp.shape[0]):
        temp_brand=temp.loc[j,'resuvatin_brand']
        if temp_brand == brand:
            continue
        else:
            count+=1
            brand=temp_brand
    temp['换药次数']=count
    temp_list.append(temp)
df_drug_vatin_before=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [870]:
df_drug_vatin_before.shape

(32058, 39)

In [871]:
df_drug_vatin_before['换药次数'].value_counts()

0     24184
1      5382
2      1718
3       349
4       189
5        74
18       42
8        28
6        28
23       25
7        23
14       16
Name: 换药次数, dtype: int64

In [872]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.4.1_政策前_换药次数.csv')

#### 政策后

In [873]:
# 计算换药次数
temp_list=[]
for i in df_drug_vatin_after.患者编号.unique():
    temp=df_drug_vatin_after[df_drug_vatin_after['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    # 统计换药次数
    count=0
    brand=temp.loc[0,'resuvatin_brand']
    for j in range(1,temp.shape[0]):
        temp_brand=temp.loc[j,'resuvatin_brand']
        if temp_brand == brand:
            continue
        else:
            count+=1
            brand=temp_brand
    temp['换药次数']=count
    temp_list.append(temp)
df_drug_vatin_after=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [874]:
df_drug_vatin_after.shape

(26243, 39)

In [875]:
df_drug_vatin_after['换药次数'].value_counts()

0     18034
1      5414
2      1944
3       475
4       164
8        58
6        51
5        44
22       23
13       18
10       18
Name: 换药次数, dtype: int64

In [876]:
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.4.2_政策后_换药次数.csv')

### 初始用药选择

In [877]:
# 政策前
df_drug_first_brand_before=df_drug_vatin_before.sort_values(['患者编号','医嘱开始时间'],ascending=True).drop_duplicates(['患者编号'],keep='first')
df_drug_first_brand_before=df_drug_first_brand_before[['患者编号','resuvatin_brand']].rename(columns={'resuvatin_brand':'初始用药选择'})
# 并入初始用药选择
df_drug_vatin_before = pd.merge(df_drug_vatin_before,df_drug_first_brand_before,on='患者编号',how='left')

In [878]:
# 政策后
df_drug_first_brand_after=df_drug_vatin_after.sort_values(['患者编号','医嘱开始时间'],ascending=True).drop_duplicates(['患者编号'],keep='first')
df_drug_first_brand_after=df_drug_first_brand_after[['患者编号','resuvatin_brand']].rename(columns={'resuvatin_brand':'初始用药选择'})
# 并入初始用药选择
df_drug_vatin_after = pd.merge(df_drug_vatin_after,df_drug_first_brand_after,on='患者编号',how='left')

In [879]:
df_drug_vatin_after.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择'],
      dtype='object')

In [880]:
print(df_drug_vatin_before.shape)
print(df_drug_vatin_after.shape)

(32058, 40)
(26243, 40)


In [881]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.5.1_政策前_初始用药选择.csv')
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.5.2_政策后_初始用药选择.csv')

In [882]:
df_drug_vatin_before=pd.read_csv(project_path+'/data/processed_data/政策性/df_3.7.5.1_政策前_初始用药选择.csv')
if 'Unnamed: 0' in df_drug_vatin_before.columns:
    del df_drug_vatin_before['Unnamed: 0']

In [883]:
df_drug_vatin_after=pd.read_csv(project_path+'/data/processed_data/政策性/df_3.7.5.2_政策后_初始用药选择.csv')
if 'Unnamed: 0' in df_drug_vatin_after.columns:
    del df_drug_vatin_after['Unnamed: 0']

In [884]:
df_drug_vatin_before.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择'],
      dtype='object')

In [885]:
df_drug_vatin_after.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择'],
      dtype='object')

### 第二种用药选择

#### 政策前

In [886]:
# 计算换药次数
temp_list=[]
for i in df_drug_vatin_before.患者编号.unique():
    temp=df_drug_vatin_before[df_drug_vatin_before['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    count=temp.loc[0,'换药次数']  # 换药次数
    brand=temp.loc[0,'初始用药选择']
    if count<1:
        temp['第二种药物选择']=np.nan
    else:
        for j in range(1,temp.shape[0]):
            temp_brand=temp.loc[j,'resuvatin_brand']
            if temp_brand == brand:
                continue
            else:
                brand=temp_brand
                break
        temp['第二种药物选择']=brand
    temp_list.append(temp)
df_drug_vatin_before=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [887]:
df_drug_vatin_before.shape

(32058, 41)

In [888]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.6.1_政策前_第二种用药选择.csv')

#### 政策后

In [889]:
# 计算换药次数
temp_list=[]
for i in df_drug_vatin_after.患者编号.unique():
    temp=df_drug_vatin_after[df_drug_vatin_after['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    count=temp.loc[0,'换药次数']  # 换药次数
    brand=temp.loc[0,'初始用药选择']
    if count<1:
        temp['第二种药物选择']=np.nan
    else:
        for j in range(1,temp.shape[0]):
            temp_brand=temp.loc[j,'resuvatin_brand']
            if temp_brand == brand:
                continue
            else:
                brand=temp_brand
                break
        temp['第二种药物选择']=brand
    temp_list.append(temp)
df_drug_vatin_after=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [890]:
df_drug_vatin_after.shape

(26243, 41)

In [891]:
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.6.2_政策后_第二种用药选择.csv')

### 第三种用药选择

#### 政策前

In [892]:
df_drug_vatin_before.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择', '第二种药物选择'],
      dtype='object')

In [893]:
temp_list=[]
for i in df_drug_vatin_before.患者编号.unique():
    temp=df_drug_vatin_before[df_drug_vatin_before['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    count=temp.loc[0,'换药次数']  # 换药次数
    first_brand=temp.loc[0,'初始用药选择']
    second_brand=temp.loc[0,'第二种药物选择']
    third_brand=np.nan
    if count>=2:
        index=temp['resuvatin_brand'].to_list().index(second_brand)
        for j in range(index+1,temp.shape[0]):
            temp_brand=temp.loc[j,'resuvatin_brand']
            if temp_brand == second_brand:
                continue
            else:
                third_brand=temp_brand
                break
    temp['第三种药物选择']=third_brand
    temp_list.append(temp)

In [894]:
df_drug_vatin_before=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [895]:
df_drug_vatin_before.shape

(32058, 42)

In [896]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.7.1_政策前_第三种用药选择.csv')

#### 政策后

In [897]:
df_drug_vatin_after.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择', '第二种药物选择'],
      dtype='object')

In [898]:
temp_list=[]
for i in df_drug_vatin_after.患者编号.unique():
    temp=df_drug_vatin_after[df_drug_vatin_after['患者编号']==i]
    temp=temp.sort_values(['医嘱开始时间'],ascending=True).reset_index(drop=True)
    count=temp.loc[0,'换药次数']  # 换药次数
    first_brand=temp.loc[0,'初始用药选择']
    second_brand=temp.loc[0,'第二种药物选择']
    third_brand=np.nan
    if count>=2:
        index=temp['resuvatin_brand'].to_list().index(second_brand)
        for j in range(index+1,temp.shape[0]):
            temp_brand=temp.loc[j,'resuvatin_brand']
            if temp_brand == second_brand:
                continue
            else:
                third_brand=temp_brand
                break
    temp['第三种药物选择']=third_brand
    temp_list.append(temp)

In [899]:
df_drug_vatin_after=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [900]:
df_drug_vatin_after.shape

(26243, 42)

In [901]:
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.7.2_政策后_第三种用药选择.csv')

### 换药标签

#### 政策前

In [902]:
df_drug_vatin_before.shape

(32058, 42)

In [903]:
df_drug_vatin_before.columns

Index(['data_source', '患者编号', '药物通用名称', '医嘱开始时间', '医嘱停止时间', '药物规格', '药物剂量',
       '药物剂量单位名称', '频率名称', '开药总数量', '开药总数量单位名称', '药物剂型', '用药时长', '医嘱开始日期',
       '日剂量', '开药总剂量', 'resuvatin_brand', '瑞舒伐他汀', '阿托伐他汀', '匹伐他汀', '氟伐他汀',
       '氨氯地平阿托伐他汀', '辛伐他汀', '洛伐他汀', '冠心病', '高脂血症', '冠心病_高脂血症', '初次用药年', '性别',
       '出生日期', '年龄', '吸烟史', '饮酒史', '高血压', '糖尿病', '首次用药时间', '末次用药时间', 'group',
       '换药次数', '初始用药选择', '第二种药物选择', '第三种药物选择'],
      dtype='object')

In [904]:
# 换药一次：京新、可定、其他汀
df_drug_vatin_before['可定_京新']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='京新')
                                                         else 0,axis=1)
df_drug_vatin_before['京新_可定']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_before['可定_其他汀']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='其他汀')
                                                         else 0,axis=1)
df_drug_vatin_before['其他汀_可定']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='其他汀') & (x['第二种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_before['京新_其他汀']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='其他汀')
                                                         else 0,axis=1)
df_drug_vatin_before['其他汀_京新']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='其他汀') & (x['第二种药物选择']=='京新')
                                                         else 0,axis=1)
# 换药2次：京新_可定_京新，可定_京新_可定
df_drug_vatin_before['可定_京新_可定']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==2) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='京新') & (x['第三种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_before['京新_可定_京新']=df_drug_vatin_before.apply(lambda x: 1 if (x['换药次数']==2) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='可定') & (x['第三种药物选择']=='京新')
                                                         else 0,axis=1)

In [905]:
df_drug_vatin_before['可定_其他汀'].value_counts()

0    29435
1     2623
Name: 可定_其他汀, dtype: int64

In [906]:
df_drug_vatin_before.shape

(32058, 50)

In [907]:
df_drug_vatin_before.to_csv(project_path+'/data/processed_data/政策性/df_3.7.8.1_政策前_换药标签.csv')

#### 政策后

In [908]:
# 换药一次：京新、可定、其他汀
df_drug_vatin_after['可定_京新']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='京新')
                                                         else 0,axis=1)
df_drug_vatin_after['京新_可定']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_after['可定_其他汀']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='其他汀')
                                                         else 0,axis=1)
df_drug_vatin_after['其他汀_可定']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='其他汀') & (x['第二种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_after['京新_其他汀']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='其他汀')
                                                         else 0,axis=1)
df_drug_vatin_after['其他汀_京新']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==1) & (x['初始用药选择']=='其他汀') & (x['第二种药物选择']=='京新')
                                                         else 0,axis=1)
# 换药2次：京新_可定_京新，可定_京新_可定
df_drug_vatin_after['可定_京新_可定']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==2) & (x['初始用药选择']=='可定') & (x['第二种药物选择']=='京新') & (x['第三种药物选择']=='可定')
                                                         else 0,axis=1)
df_drug_vatin_after['京新_可定_京新']=df_drug_vatin_after.apply(lambda x: 1 if (x['换药次数']==2) & (x['初始用药选择']=='京新') & (x['第二种药物选择']=='可定') & (x['第三种药物选择']=='京新')
                                                         else 0,axis=1)

In [909]:
df_drug_vatin_after['可定_京新'].value_counts()

0    24815
1     1428
Name: 可定_京新, dtype: int64

In [910]:
df_drug_vatin_after.shape

(26243, 50)

In [911]:
df_drug_vatin_after.to_csv(project_path+'/data/processed_data/政策性/df_3.7.8.2_政策后_换药标签.csv')

### 取第一条数据

In [912]:
# 政策前
df_drug_vatin_before_first=df_drug_vatin_before.sort_values(['患者编号','医嘱开始时间'],ascending=True).drop_duplicates(['患者编号'],keep='first').reset_index(drop=True)

In [913]:
df_drug_vatin_before_first.shape

(7018, 50)

In [914]:
# 政策后
df_drug_vatin_after_first=df_drug_vatin_after.sort_values(['患者编号','医嘱开始时间'],ascending=True).drop_duplicates(['患者编号'],keep='first').reset_index(drop=True)

In [915]:
df_drug_vatin_after_first.shape

(5856, 50)

In [916]:
df_drug_vatin_before_first.to_csv(project_path+'/data/processed_data/政策性/df_3.7.9.1_政策前_第一条换药标签.csv')
df_drug_vatin_after_first.to_csv(project_path+'/data/processed_data/政策性/df_3.7.9.2_政策后_第一条换药标签.csv')

## 医保类型

### 人民医院

In [242]:
# 人民医院--检验数据
df_insurance_1 = pd.read_excel(r'./data/raw_data/人民医院/6-医疗付费方式.xlsx')

In [245]:
df_insurance_1.columns

Index(['患者编号', '当前就诊次数/命中就诊总次数', '医疗付费方式'], dtype='object')

In [246]:
df_insurance_1=df_insurance_1[['患者编号', '医疗付费方式']].rename(columns={'医疗付费方式':'医保类型'})

In [247]:
df_insurance_1.head()

,患者编号,医保类型
0,pat_0d9297a346707bce01f0b4637484efe9,全自费
1,pat_0ebd83a8c3990a103016a650f84616f8,商业医疗保险
2,pat_13da443c402ad02c9e9de0c5c0751eda,商业医疗保险
3,pat_13da443c402ad02c9e9de0c5c0751eda,商业医疗保险
4,pat_13da443c402ad02c9e9de0c5c0751eda,商业医疗保险


### 世纪坛医院

In [243]:
# 世纪坛医院--检验数据
df_insurance_2 = pd.read_excel(r'./data/raw_data/首都医科大学附属北京世纪坛医院—阿托他汀临床疗效及安全性评价的真实世界研究-住院+门诊-2018.3.23-2020.3.xlsx',
                           sheet_name='病历')
df_insurance_2 = df_insurance_2.dropna(how='all',axis=1)

In [244]:
df_insurance_2.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'patient_SN', '性别', '出生日期', '年龄', '民族',
       '就诊次数（住院）', '就诊时间（门诊）', '就诊科室（门诊）', '就诊类型', '入院(就诊)时间', '既往是否有疾病史',
       '历史总门诊就诊次数', '历史总急诊就诊次数', '历史总住院就诊次数', '初步诊断（首次病程）', '医保类型', '入院（就诊）科室',
       '住院天数'],
      dtype='object')

In [ ]:
df_insurance_2=df_insurance_2[['patient_SN','医保类型', '入院（就诊）科室']]
# 置空噪声数据
df_insurance_2['医保类型']=df_insurance_2['医保类型'].apply(lambda x: np.nan if '科' in x else x)
df_insurance_2['入院（就诊）科室']=df_insurance_2['入院（就诊）科室'].apply(lambda x: np.nan if '科' in x else x)

In [255]:
# 将入院（就诊）科室为空的数据填充为同行的医保类型
df_insurance_2['入院（就诊）科室']=df_insurance_2.apply(lambda x: x['医保类型'] if pd.isnull(x['入院（就诊）科室']) else x['入院（就诊）科室'],axis=1)

In [261]:
df_insurance_2.columns=['患者编号','就诊科室','医保类型']
df_insurance_2=df_insurance_2[['患者编号','医保类型']]

In [262]:
df_insurance_2.columns

Index(['患者编号', '医保类型'], dtype='object')

In [256]:
df_insurance_2.to_csv(project_path+'/data/processed_data/依从性/df_temp_医保类型_世纪坛.csv')

### 合并医保类型

In [290]:
df_insurance=pd.concat([df_insurance_1,df_insurance_2],axis=0)
# 删除重复
df_insurance=df_insurance.drop_duplicates(['患者编号', '医保类型'],keep='first')
# 删除其他医保类型
df_insurance=df_insurance[~ df_insurance['医保类型'].str.contains('其他')]

In [291]:
# 合并同一人的多种医保类型
temp_list=[]
for i in df_insurance.患者编号.unique():
    temp=df_insurance[df_insurance['患者编号']==i].reset_index(drop=True)
    if temp.shape[0]>1:
        temp=temp.loc[:0,]
        temp.loc[0,'医保类型']=';'.join(temp['医保类型'].to_list())
    temp_list.append(temp)

In [292]:
df_insurance=pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [293]:
df_insurance

,患者编号,医保类型
0,pat_0d9297a346707bce01f0b4637484efe9,全自费
1,pat_0ebd83a8c3990a103016a650f84616f8,商业医疗保险
2,pat_13da443c402ad02c9e9de0c5c0751eda,商业医疗保险
3,pat_2c40bef1b35d17984b1d393c6be55688,全自费
4,pat_0f6bf632666460422c4923f591ede61f,全自费
...,...,...
60719,ffe25ffb99f78a2d9ac99af9b6450cca,城镇职工基本医疗保险
60720,ffe2c486261b6f2a033b31e6db20d2c6,全公费
60721,ffe72aa2d2eb81cb9e8da3cc10103578,城镇职工基本医疗保险
60722,fff5ec2c3a7757680d84c1f7d45b065f,全自费


In [294]:
df_insurance.患者编号.value_counts()

71939fa186293b70658710ba8dcfdbc7        1
pat_7e072e7fcfc30bf5a6ee6423eb328442    1
f462e1be4f390322dda38c96bf68e22d        1
pat_a6c6a922da266a9beb4a18b21a13d436    1
pat_a4446be86e5b7fd63a6f456fe22311f3    1
                                       ..
897117b04de66d4442e9e87d412bf55d        1
35a65894d5917bea251724284b590e3b        1
pat_cd508afce99f29a08c9e17dec942b102    1
pat_b163cadd8c03b256fc985511d1ba8237    1
pat_ccf06e11a32001e902146a267900858b    1
Name: 患者编号, Length: 60724, dtype: int64

### 并入医保类型

In [407]:
# 政策前观察期
df_drug_resuvatin_keep_before=pd.merge(df_drug_resuvatin_keep_before,df_insurance,on='患者编号',how='left')
#政策后观察期
df_drug_resuvatin_keep_after=pd.merge(df_drug_resuvatin_keep_after,df_insurance,on='患者编号',how='left')

In [408]:
df_drug_resuvatin_keep_before.to_csv(project_path+'/data/processed_data/依从性/df_3.7.4_政策前_并入医保类型.csv')
df_drug_resuvatin_keep_after.to_csv(project_path+'/data/processed_data/依从性/df_3.7.4_政策后_并入医保类型.csv')